In [17]:
# ! pip install pandas more-itertools scikit-learn geopy haversine folium

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import sys, os
sys.path.append(os.getcwd())

In [30]:
DATA_DIR = "/home/jovyan/work/dataX"
N_POINTS_TO_SHOW = 100
MAX_REQUESTS=300
GRID_SPACING_M = 100
RADIUS_SIZE_KM = 9
CENTER_POINT = (52.179363, 21.030999)
#(52.179574895184885, 21.022671986482695)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import json
from pathlib import Path

from xplore.location_history import read_location_history_dir, read_points_excluded_from_exploration
from xplore.road_points import get_grid_and_road_points
from xplore.xplore import create_knn, get_non_visited_road_points, show_map_with_points

In [5]:
visited_points = read_location_history_dir(DATA_DIR)

json loaded...
read 1003513 unique locations


(1003513, 2)

In [33]:
points_excluded_from_exploration = read_points_excluded_from_exploration(DATA_DIR, GRID_SPACING_M)

In [35]:
knn = create_knn(visited_points, points_excluded_from_exploration)

In [9]:
GOOGLE_API_KEY = json.load(open(Path(DATA_DIR) / "gapi_key.json", "r"))["key"]
grid_and_road_points = get_grid_and_road_points(CENTER_POINT, RADIUS_SIZE_KM, GRID_SPACING_M, DATA_DIR, MAX_REQUESTS, GOOGLE_API_KEY)

grid from (52.179363, 21.030999) with radius 9 km and spacing 100 m has 25440 points
have cached grid points (25440) and cached road points (18054)
needs 0 points to be mapped to roads
after mapping have grid points (25440) and road points (18054)


In [36]:
non_visited_points = get_non_visited_road_points(knn, grid_and_road_points["road_points"], CENTER_POINT, RADIUS_SIZE_KM, GRID_SPACING_M)

have 6458 non-visited points


In [37]:
show_map_with_points(CENTER_POINT, non_visited_points[:N_POINTS_TO_SHOW])